In [1]:
#get the goods

import geopandas as gpd
import pandas as pd
import fiona

In [2]:
#populating data for use in subsequent steps

layers = fiona.listlayers('../lab1/lab1.gpkg')
print(layers)

['soilmu_a_co001', 'soilmu_a_co618', 'soilmu_a_co641', 'soilmu_a_co642', 'soilmu_a_co643', 'soilmu_a_co644', 'soilmu_a_co645', 'soilmu_a_co651', 'soilmu_a_co653', 'wbdhu8_lab1', 'muaggatt_co001', 'muaggatt_co618', 'muaggatt_co641', 'muaggatt_co642', 'muaggatt_co643', 'muaggatt_co644', 'muaggatt_co645', 'muaggatt_co651', 'muaggatt_co653']


In [3]:
#creates an empty list, then fills those lists with the feature classes and the data tables so we can use them in the for loop

survey_plg = [] 
data_tbl = []

for layer_name in layers:
    if layer_name.startswith('soilmu'):
        survey_plg.append(layer_name)

for layer_name in layers:
    if layer_name.startswith('muaggatt'):
        data_tbl.append(layer_name)

print(survey_plg)
print(data_tbl)

['soilmu_a_co001', 'soilmu_a_co618', 'soilmu_a_co641', 'soilmu_a_co642', 'soilmu_a_co643', 'soilmu_a_co644', 'soilmu_a_co645', 'soilmu_a_co651', 'soilmu_a_co653']
['muaggatt_co001', 'muaggatt_co618', 'muaggatt_co641', 'muaggatt_co642', 'muaggatt_co643', 'muaggatt_co644', 'muaggatt_co645', 'muaggatt_co651', 'muaggatt_co653']


In [4]:
#matching the survey polygons to the data tables based on the last 5 characters of the file name and assigning them to new varibles
#then telling the program to read the polygons and tables so magic can happen
#weird geometry column happening in the tables that needed to be dropped so the intersect later in the script can happen
#merging each feature class to table on common column
#adding a new field 'mapunit' to each merged feature class/table
#appending the merge to the empty list
#now there is all 9 merged files in a list

merged_fcs = []

for feature_class in survey_plg:
    for table in data_tbl:
        if feature_class[-5:] == table[-5:]:
            p = gpd.read_file('../lab1/lab1.gpkg', layer = feature_class)
            t = gpd.read_file('../lab1/lab1.gpkg', layer = table)
            t = t.drop(columns = 'geometry')
            merge = pd.merge(p, t, left_on = 'MUSYM', right_on = 'musym')
            merge['mapunit'] = feature_class[-5:]
            merged_fcs.append(merge)
            
print(merged_fcs)

[      SPATIALVER MUSYM   MUKEY  Shape_Length  Shape_Area  \
0            4.0   AaB   94035      0.010144    0.000004   
1            4.0   AaB   94035      0.011057    0.000004   
2            4.0   AaB   94035      0.010620    0.000005   
3            4.0   AaB   94035      0.015856    0.000013   
4            4.0   AaB   94035      0.025435    0.000025   
...          ...   ...     ...           ...         ...   
2637         4.0   StB   94076      0.083308    0.000224   
2638         4.0   StB   94076      0.031943    0.000038   
2639         4.0    Ro   94069      0.024232    0.000015   
2640         4.0   DUM  822616      0.022173    0.000027   
2641         4.0   HlD   94055      0.094671    0.000191   

                                               geometry musym aws025wta  \
0     MULTIPOLYGON (((-104.42220 39.93400, -104.4248...   AaB      4.15   
1     MULTIPOLYGON (((-104.27200 39.78590, -104.2749...   AaB      4.15   
2     MULTIPOLYGON (((-104.34210 39.89100, -104.3428.

In [5]:
#joining the 9 merged feature classes into a single file

joined_fc = pd.concat(merged_fcs)

print(joined_fc)

     SPATIALVER MUSYM   MUKEY  Shape_Length  Shape_Area  \
0           4.0   AaB   94035      0.010144    0.000004   
1           4.0   AaB   94035      0.011057    0.000004   
2           4.0   AaB   94035      0.010620    0.000005   
3           4.0   AaB   94035      0.015856    0.000013   
4           4.0   AaB   94035      0.025435    0.000025   
..          ...   ...     ...           ...         ...   
405         3.0    49  512721      0.013377    0.000009   
406         3.0    15  512687      0.057632    0.000130   
407         3.0    11  512683      0.072986    0.000325   
408         3.0    26  512698      0.030541    0.000027   
409         3.0    64  512736      0.012214    0.000007   

                                              geometry musym aws025wta  \
0    MULTIPOLYGON (((-104.42220 39.93400, -104.4248...   AaB      4.15   
1    MULTIPOLYGON (((-104.27200 39.78590, -104.2749...   AaB      4.15   
2    MULTIPOLYGON (((-104.34210 39.89100, -104.3428...   AaB      4.1

In [6]:
#need to read watershed layer first
#intersect the joined fc with the watershed boundaries

watershed = gpd.read_file('../lab1/lab1.gpkg', layer = 'wbdhu8_lab1')

intersect = gpd.overlay(joined_fc, watershed, how = 'intersection')

print(intersect)

      SPATIALVER MUSYM   MUKEY  Shape_Length_1  Shape_Area_1 musym aws025wta  \
0            4.0   ReD   94068        0.027154      0.000019   ReD      4.15   
1            4.0   ReD   94068        0.009159      0.000004   ReD      4.15   
2            4.0   ReD   94068        0.009860      0.000005   ReD      4.15   
3            4.0   ReD   94068        0.010011      0.000005   ReD      4.15   
4            4.0   ReD   94068        0.009844      0.000005   ReD      4.15   
...          ...   ...     ...             ...           ...   ...       ...   
4047         3.0    21  506401        0.016249      0.000015    21       1.3   
4048         3.0    23  506451        0.062679      0.000170    23      2.11   
4049         3.0    23  506451        0.024121      0.000036    23      2.11   
4050         3.0    39  506445        0.196200      0.000692    39      3.25   
4051         3.0    17  506399        0.014428      0.000011    17      3.75   

     aws0150wta                    drcl

In [7]:
#finding the number of features in each watershed

result = intersect.groupby(['NAME'])['mapunit'].count()

print(result)

NAME
Big Thompson    1979
St. Vrain       2073
Name: mapunit, dtype: int64
